In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [48]:
# 读取数据
tmp = pd.read_csv('./left_feature.csv')

In [50]:
tmp.set_indexdex('vid', inplace=True)

In [53]:
tmp.drop(['Unnamed: 0'], axis=1, inplace=True)

In [55]:
# 读取train和并与tmp合并
train_df=pd.read_csv('./input/meinian_round1_train_20180408.csv',sep=',',encoding='gbk')

# 对train集的label进行清洗
def clean_label(x):
    x=str(x)
    if '+' in x:#16.04++
        i=x.index('+')
        x=x[0:i]
    if '>' in x:#> 11.00
        i=x.index('>')
        x=x[i+1:]
    if len(x.split(sep='.'))>2:#2.2.8
        i=x.rindex('.')
        x=x[0:i]+x[i+1:]
    if '未做' in x or '未查' in x or '弃查' in x or '不查' in x or '弃检' in x:
        x=np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x=x[0:4]
    return x

def clean_label(x):
    x=str(x)
    if '+' in x:#16.04++
        i=x.index('+')
        x=x[0:i]
    if '>' in x:#> 11.00
        i=x.index('>')
        x=x[i+1:]
    if len(x.split(sep='.'))>2:#2.2.8
        i=x.rindex('.')
        x=x[0:i]+x[i+1:]
    if '未做' in x or '未查' in x or '弃查' in x or '不查' in x or '弃检' in x:
        x=np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x=x[0:4]
    return x
    
def data_clean(df):
    label_list = ['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白']
    for c in label_list:
        df[c] = df[c].apply(clean_label)
        df[c] = df[c].astype('float64')
    df.dropna(axis=0, inplace=True, how='any')

    return df

train_df = data_clean(train_df)
# 设置相同的index方便合并
train_df.set_index('vid',inplace=True)
anlyse_train = pd.concat([tmp, train_df], axis=1, join='inner')

# 布尔型相关的函数

In [57]:
# 纯文本枚举型，先分析均值，再根据关键词提特征

# 分析关键词均值的函数
def analy2(df, col, in_str):
    label_list = ['收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']
    
    def has_str(in_str):
        def has_str_apply(x):
            x = str(x)
            if in_str in x:
                return in_str
            else:
                return 0
        return has_str_apply
    
    df_tmp = df[[col] + label_list]
    df_tmp[col] = df_tmp[col].apply(has_str(in_str))
    df_tmp = df_tmp.groupby(df_tmp[col])
    cnt = df_tmp.count().values[:,1]
    df_output = df_tmp.mean()
    df_output['counts'] = cnt
    print(df_output)
    return df_output

def label_apply(label_list):
    def label_list_apply(x):
        x = str(x)
        for l in label_list:
            if (l in x):
                return 1
        return 0
    return label_list_apply

# 提取特征放在后面的函数：
def add_feature(df1, df2, col, label_list, new_feature_name):
    tmp_ = df2[col].apply(label_apply(label_list)).values
    df1[new_feature_name] = tmp_
    return df1

def analys(df, col):
    label_list = ['收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']
    df_tmp = df[[col] + label_list]
    df_tmp = df_tmp.groupby(df_tmp[col])
    cnt = df_tmp.count().values[:,1]
    df_output = df_tmp.mean()
    df_output['counts'] = cnt
    print(df_output)
    return df_output

# 枚举型相关的函数

In [58]:
def oh_apply(label_list):
    def label_list_apply(x):
        x = str(x)
        for i in range(1, len(label_list)+1):
            tmp_l = label_list[i-1]
            for l in tmp_l:
                if (l in x):
                    return i
        return 0
    return label_list_apply

# 提取特征放在后面的函数：
def add_oh_feature(df1, df2, col, label_list, new_feature_name):
    tmp_ = df2[col].apply(oh_apply(label_list)).values
    df1[new_feature_name] = tmp_
    return df1

# 分析方法：

In [75]:
# 观察 这一列中有多少不同的特征，看它们的均值，最后是数量
a = analys(anlyse_train, tmp.columns[121])
a.to_csv('test.csv')

          收缩压        舒张压  血清甘油三酯  血清高密度脂蛋白  血清低密度脂蛋白  counts
069018                                                      
0.0     134.0  77.000000    0.63  1.630000  4.230000       1
+       128.0  79.000000    5.75  1.110000  2.420000       1
-       114.0  73.166667    1.40  1.258333  2.636667       6
0       130.0  78.000000    1.05  1.560000  4.290000       1


In [76]:
# 如果种类太多，用关键词分析, 看存在这个关键词和不纯在这个关键的均值对比
tmp = analy2(anlyse_train, tmp.columns[121], '+')

               收缩压        舒张压    血清甘油三酯  血清高密度脂蛋白  血清低密度脂蛋白  counts
069018                                                             
0       126.049567  79.642246  1.616537  1.406715  2.769669   38191
+       128.000000  79.000000  5.750000  1.110000  2.420000       1
